In [22]:
import time
import os
import sys
sys.path.append(r'/home/jamesrowland/Documents/Code/blimp')
from utils.parse_markpoints import ParseMarkpoints
from utils import mat_loader as ml
import scipy.io
import yaml
import numpy as np
import imageio
import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
# output sampling rate of naparm
naparm_rate = 1000

naparm_path = r'/home/jamesrowland/Desktop/2019-02-04_J007_NAPARM_001'


with open('/home/jamesrowland/Documents/Code/blimp/blimp_settings.yaml', 'r') as stream:
            yaml_dict = yaml.load(stream)


In [3]:
for file in os.listdir(naparm_path):
    if file.endswith('.xml'):
        xml_path= os.path.join(naparm_path, file)
    elif file.endswith('gpl'):
        gpl_path = os.path.join(naparm_path, file)
    elif file.endswith('Points.mat'):
        points_path = os.path.join(naparm_path, file)
    elif file == 'ToPV_AllTrials.dat':
        to_pv = os.path.join(naparm_path, file)
    elif file == 'ToSLM_AllTrials.dat':
        to_slm = os.path.join(naparm_path, file)
        
        
       
tiff_list = []
for file in os.listdir(os.path.join(naparm_path, 'PhaseMasks')):
    if file.endswith('.tif') or file.endswith('.tiff'):
        tiff_list.append(os.path.join(naparm_path, 'PhaseMasks', file))
        

mask_list = [imageio.imread(tiff) for tiff in tiff_list]
pm = ParseMarkpoints(xml_path=xml_path, gpl_path=gpl_path)      



In [11]:
s = ml.load_mat_struct(points_path)

points = s['points']

all_x_galvo = points.GroupCentroidX
all_y_galvo = points.GroupCentroidY

groupings = points.Group
num_groups = max(groupings)

galvo_x = []
galvo_y = []


for group in range(1,num_groups+1):
    
    group_idx = np.where(groupings==group)
    
    # xna dy galvo positions of all points in pixels
    x_px = all_x_galvo[group_idx]
    y_px = all_y_galvo[group_idx]
    
    # all elements should be the same (stolen from stack overflow)
    assert list(x_px).count(x_px[0]) == len(x_px)
    assert list(y_px).count(y_px[0]) == len(y_px)
    
    #into foxy ratio format
    galvo_x.append(x_px[0] / 512)
    galvo_y.append(y_px[0] / 512)

assert len(galvo_x) == len(galvo_y) == len(pm.durations) == len(pm.laser_powers)


In [5]:
pv_arr = np.fromfile(to_pv, dtype=float)
slm_arr = np.fromfile(to_slm, dtype=float)

#time between each SLM trigger
slm_diff = np.diff(np.where(np.diff(slm_arr) > 0))[0] / naparm_rate * 1000 #ms 

#the time between each PV trigger
pv_diff = np.diff(np.where(np.diff(pv_arr) > 0))[0] / naparm_rate * 1000  #ms



In [38]:
mp_strings = []

#currently only supporting evenly spaced groups
inter_group_interval = slm_diff[0]

spiral_size = yaml_dict['spiral_size'] / (yaml_dict['FOVsize_UM_1x'] / yaml_dict['zoom'])
spiral_revolutions = pm.spiral_revolutions[0]

for group in range(num_groups):
    x = galvo_x[group]
    y = galvo_y[group]
    duration = pm.durations[group]
    laser_power = pm.laser_powers[group]
    num_spirals = pm.repetitions[group]
    
    mp_string = pm.build_strings(X=x,Y=y,duration=duration,laser_power=laser_power, \
                                 is_spiral=True, spiral_size=spiral_size, \
                                 spiral_revolutions=spiral_revolutions, num_spirals=num_spirals)

    mp_strings.append(mp_string)
    
    
all_groups_string = pm.groups_strings(inter_group_interval=inter_group_interval, \
                                      group_list=mp_strings, SLM_trigger=True)

In [45]:
from pyfiglet import Figlet
f = Figlet(font='invita')
print (f.renderText('AUTO NAPARM'))

   _____   __     __) ______) ___  
  (, /  | (, /   /   (, /   /(,  ) 
    /---|   /   /      /   /    /  
 ) /    |_ /   /    ) /   /    /   
(_/       (___(_   (_/   (___ /    
                                   
   __     __) _____   _____    _____    _____    __     __) 
  (, /|  /   (, /  | (, /   ) (, /  |  (, /   ) (, /|  /|   
    / | /      /---|  _/__ /    /---|    /__ /    / | / |   
 ) /  |/    ) /    |_ /      ) /    |_) /   \_ ) /  |/  |_  
(_/   '    (_/     ) /      (_/      (_/      (_/   '       
                  (_/                                       

